# ETL For Project 4 - Fake News Detection

Import Dependencies

In [14]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from api_keys import postgres_p
import datetime

Set Path For CSV Files

In [2]:
csv_test = os.path.join('..', 'Dataset', 'test.csv')
csv_train = os.path.join('..', 'Dataset', 'train.csv')

Load CSV Files

In [3]:
test_df = pd.read_csv(csv_test)
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [4]:
train_df = pd.read_csv(csv_train)
train_df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Convert Date Column Data Type

In [5]:
train_df.shape

(20800, 5)

In [6]:
test_df.shape

(5200, 4)

In [7]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
test_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [9]:
train_clean = train_df.fillna('')
test_clean = test_df.fillna('')

In [10]:
train_clean.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
test_clean.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [12]:
train_clean['text_corpus'] = train_clean['author']+' '+train_clean['title']+' '+train_clean['text']

In [13]:
print(train_clean['text_corpus'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: text_corpus, Length: 20800, dtype: object


In [58]:
article_id_df = pd.DataFrame(train_clean[['id', 'label']])
article_id_df.rename(columns= {'id':'article_id', 'label':'article_label'}, inplace=True)
article_id_df

,article_id,article_label
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
20795,20795,0
20796,20796,0
20797,20797,0
20798,20798,1


In [72]:
# title
title_df = pd.DataFrame(train_clean[['id', 'title']])
title_df.rename(columns= {'id':'article_id', 'title':'article_title'}, inplace=True)
title_df

,article_id,article_title
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired
3,3,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...
...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [74]:
# author
author_df = pd.DataFrame(train_clean[['id', 'author']])
author_df.rename(columns= {'id':'article_id', 'author':'article_author'}, inplace=True)
author_df

,article_id,article_author
0,0,Darrell Lucus
1,1,Daniel J. Flynn
2,2,Consortiumnews.com
3,3,Jessica Purkiss
4,4,Howard Portnoy
...,...,...
20795,20795,Jerome Hudson
20796,20796,Benjamin Hoffman
20797,20797,Michael J. de la Merced and Rachel Abrams
20798,20798,Alex Ansary


In [75]:
# text
text_df = pd.DataFrame(train_clean[['id', 'text']])
text_df.rename(columns= {'id':'article_id', 'text':'article_text'}, inplace=True)
text_df

,article_id,article_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,Ever get the feeling your life circles the rou...
2,2,"Why the Truth Might Get You Fired October 29, ..."
3,3,Videos 15 Civilians Killed In Single US Airstr...
4,4,Print \r\nAn Iranian woman has been sentenced ...
...,...,...
20795,20795,Rapper T. I. unloaded on black celebrities who...
20796,20796,When the Green Bay Packers lost to the Washing...
20797,20797,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [59]:
text_corpus_df = pd.DataFrame(train_clean[['id', 'text_corpus']])
text_corpus_df.rename(columns= {'id':'article_id', 'label':'article_label'}, inplace=True)
text_corpus_df.head()

,article_id,text_corpus
0,0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Consortiumnews.com Why the Truth Might Get You...
3,3,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Howard Portnoy Iranian woman jailed for fictio...


In [96]:
# Create a date table for article dates - for future analysis and modeling purposes
article_date = pd.DataFrame(train_clean['id'])

In [99]:
# assign existing articles todays date
article_date = article_date.assign(Date=datetime.datetime.now().date())

In [101]:
article_date.rename(columns= {'id':'article_id', 'Date':'article_date'}, inplace=True)
article_date.head()

,article_id,article_date
0,0,2023-03-19
1,1,2023-03-19
2,2,2023-03-19
3,3,2023-03-19
4,4,2023-03-19


In [102]:
protocol = 'postgresql'
username = 'postgres'
password = postgres_p

host = 'localhost'

port = 5432

database_name = 'Project_4'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

insp.get_table_names()

['article_id',
 'article_title',
 'article_author',
 'article_text',
 'article_date',
 'user_submit',
 'text_corpus']

In [103]:
# load to database
# Article_ID Table
article_id_df.to_sql(name='article_id', con=engine, if_exists='append', index=False)
# ['article_author',
author_df.to_sql(name='article_author', con=engine, if_exists='append', index=False)
#  'article_title',
title_df.to_sql(name='article_title', con=engine, if_exists='append', index=False)
#  'article_text',
text_df.to_sql(name='article_text', con=engine, if_exists='append', index=False)
#  'article_date',
article_date.to_sql(name='article_date', con=engine, if_exists='append', index=False)
# # Text_Corpus Table
text_corpus_df.to_sql(name='text_corpus', con=engine, if_exists='append', index=False)

In [104]:
pd.read_sql_query('select * from article_id', con=engine).head(15)

,article_id,article_label
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
5,5,0
6,6,1
7,7,0
8,8,0
9,9,0


In [105]:
pd.read_sql_query('select * from article_author', con=engine).head(15)

,article_id,article_author
0,0,Darrell Lucus
1,1,Daniel J. Flynn
2,2,Consortiumnews.com
3,3,Jessica Purkiss
4,4,Howard Portnoy
5,5,Daniel Nussbaum
6,6,
7,7,Alissa J. Rubin
8,8,
9,9,Megan Twohey and Scott Shane


In [106]:
pd.read_sql_query('select * from article_text', con=engine).head(15)

,article_id,article_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,Ever get the feeling your life circles the rou...
2,2,"Why the Truth Might Get You Fired October 29, ..."
3,3,Videos 15 Civilians Killed In Single US Airstr...
4,4,Print \r\nAn Iranian woman has been sentenced ...
5,5,"In these trying times, Jackie Mason is the Voi..."
6,6,Ever wonder how Britain’s most iconic pop pian...
7,7,"PARIS — France chose an idealistic, traditi..."
8,8,Donald J. Trump is scheduled to make a highly ...
9,9,A week before Michael T. Flynn resigned as nat...


In [107]:
pd.read_sql_query('select * from article_date', con=engine).head(15)

,article_id,article_date
0,0,2023-03-19
1,1,2023-03-19
2,2,2023-03-19
3,3,2023-03-19
4,4,2023-03-19
5,5,2023-03-19
6,6,2023-03-19
7,7,2023-03-19
8,8,2023-03-19
9,9,2023-03-19


In [108]:
pd.read_sql_query('select * from article_title', con=engine).head(15)

,article_id,article_title
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired
3,3,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...
5,5,Jackie Mason: Hollywood Would Love Trump if He...
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...
7,7,Benoît Hamon Wins French Socialist Party’s Pre...
8,8,Excerpts From a Draft Script for Donald Trump’...
9,9,"A Back-Channel Plan for Ukraine and Russia, Co..."


In [109]:
pd.read_sql_query('select * from text_corpus', con=engine).head(15)

,article_id,text_corpus
0,0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Consortiumnews.com Why the Truth Might Get You...
3,3,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Howard Portnoy Iranian woman jailed for fictio...
5,5,Daniel Nussbaum Jackie Mason: Hollywood Would ...
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite...
7,7,Alissa J. Rubin Benoît Hamon Wins French Socia...
8,8,Excerpts From a Draft Script for Donald Trump...
9,9,Megan Twohey and Scott Shane A Back-Channel Pl...
